### OpenCV的简单旋转问题

图像在旋转时会被“剪切”，整个图像不会保留在视场中。

In [422]:
import cv2

image = cv2.imread('pill_01.png')
(h, w, d) = image.shape

center = (w // 2, h // 2)

# 以图片中心点为转轴，顺时针旋转45度，放缩比例为1.0，得到旋转矩阵
M = cv2.getRotationMatrix2D(center, -45, 1.0)

# 将原图片像素坐标按照旋转矩阵做仿射运算，得到旋转后的像素坐标，并指定返回图片显示范围 [0 ~ w, 0 ~ h]
rotated = cv2.warpAffine(image, M, (w, h))

cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey()
cv2.destroyAllWindows()

### 实施不会切断图像的旋转功能

In [423]:
import numpy as np

# 转轴对结果没有影响，不用传参
def getNewRotationMatrix2D(w, h, angle, scale):
    M = cv2.getRotationMatrix2D((0, 0), angle, scale)
    points = np.array([[0, 0], [w, 0], [0, h], [w, h]])
    nPoints = (np.matmul(M[:, :2], points.T) + M[:, 2:]).astype(np.int).T
    (x, y, w, h) = cv2.boundingRect(nPoints)
    M[0, 2] -= x
    M[1, 2] -= y
    return M, (w, h)

(h, w, d) = image.shape

M, (nW, nH) = getNewRotationMatrix2D(w, h, -45, 1.0)
rotated = cv2.warpAffine(image, M, (nW, nH))

cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey()
cv2.destroyAllWindows()

转换为灰度，使其模糊，并应用边缘检测以显示药丸的轮廓：

In [424]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
edged = cv2.Canny(gray, 20, 100)

cv2.imshow("edged", edged)
cv2.waitKey()
cv2.destroyAllWindows()

应用轮廓检测来找到药丸的轮廓：

In [425]:
# 轮廓列表
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 最大轮廓
maxContour = max(cnts[0], key=cv2.contourArea)

### 根据最大轮廓展示图像的旋转功能

In [426]:
import numpy as np

# 转轴对结果没有影响，不用传参
def getNew2RotationMatrix2D(contour, angle, scale):
    M = cv2.getRotationMatrix2D((0, 0), angle, scale)
    contour = contour.squeeze()
    nContour = (np.matmul(M[:, :2], contour.T) + M[:, 2:]).astype(np.int).T
    (x, y, w, h) = cv2.boundingRect(nContour)
    M[0, 2] -= x
    M[1, 2] -= y
    return M, (w, h)

M, (nW, nH) = getNew2RotationMatrix2D(maxContour, -45, 1.0)
rotated = cv2.warpAffine(image, M, (nW, nH))

cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey()
cv2.destroyAllWindows()

从图像中提取药丸的ROI：

In [427]:
# 抓住最大轮廓，然后为药丸画一个面具
mask = np.zeros(gray.shape, dtype="uint8")
cv2.drawContours(mask, [maxContour], -1, 255, -1)
# 计算药丸的边界框，然后提取ROI，并用面具遮盖
(x, y, w, h) = cv2.boundingRect(maxContour)
imageROI = image[y:y + h, x:x + w]
maskROI = mask[y:y + h, x:x + w]
imageROI = cv2.bitwise_and(imageROI, imageROI, mask=maskROI)

cv2.imshow("maskROI", maskROI)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imshow("imageROI", imageROI)
cv2.waitKey()
cv2.destroyAllWindows()

### 使用三种旋转方式，分别进行360度周转

In [428]:
(h, w, d) = imageROI.shape
center = (w // 2, h // 2)
    
for angle in np.arange(0, 360, 15):
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(imageROI, M, (w, h))
    
    cv2.imshow("Rotated (Problematic)", rotated)
    cv2.waitKey(100)
    
cv2.destroyAllWindows()

In [429]:
(h, w, d) = imageROI.shape
    
for angle in np.arange(0, 360, 15):
    M, (nW, nH) = getNewRotationMatrix2D(w, h, angle, 1.0)
    rotated = cv2.warpAffine(imageROI, M, (nW, nH))
    
    cv2.imshow("Rotated (Correct)", rotated)
    cv2.waitKey(100)

cv2.destroyAllWindows()

In [433]:
newContour = maxContour.copy()
(x, y, w, h) = cv2.boundingRect(newContour)
newContour[:, :, 0] -= x
newContour[:, :, 1] -= y

for angle in np.arange(0, 360, 15):
    M, (nW, nH) = getNew2RotationMatrix2D(newContour, angle, 1.0)
    rotated = cv2.warpAffine(imageROI, M, (nW, nH))
    
    cv2.imshow("Rotated (Correct)", rotated)
    cv2.waitKey(100)

cv2.destroyAllWindows()